# Spark DataFrame

In [101]:
#import paskages
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when

In [102]:
# initializes a Spark session
spark= SparkSession.builder.appName("demo").getOrCreate()

In [103]:
#Creating a DataFrame
df=spark.createDataFrame([
    ("Karam",23 ,450),
    ("khaled",16 ,220),
    ("Omar",31,520),
    ("Ahmad",17,250),
    ("Ali",22,410),
    ("Hasn",10 , 0) ],
    ['name','age','salary']
)


In [104]:
#show data
df.show()

+------+---+------+
|  name|age|salary|
+------+---+------+
| Karam| 23|   450|
|khaled| 16|   220|
|  Omar| 31|   520|
| Ahmad| 17|   250|
|   Ali| 22|   410|
|  Hasn| 10|     0|
+------+---+------+



In [105]:
#print Schema
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- salary: long (nullable = true)



###### Add a Column

In [106]:
#adds a new column '' based on conditions regarding the 'age' column.
df_1= df.withColumn(
        "life_stage",
   when(col('age')<13 ,"child")
    .when(col('age').between(13,18),"teenager"  )
    .when(col('age').between(18,24) ,"young")
    .otherwise('adult'),                 
    )

In [107]:
df_1.show()

+------+---+------+----------+
|  name|age|salary|life_stage|
+------+---+------+----------+
| Karam| 23|   450|     young|
|khaled| 16|   220|  teenager|
|  Omar| 31|   520|     adult|
| Ahmad| 17|   250|  teenager|
|   Ali| 22|   410|     young|
|  Hasn| 10|     0|     child|
+------+---+------+----------+



In [108]:
#display salary aftrer add bonus 0.20%
df_1.withColumn("salary_bonus_0.20%",df_1['salary']*1.20).show()

+------+---+------+----------+------------------+
|  name|age|salary|life_stage|salary_bonus_0.20%|
+------+---+------+----------+------------------+
| Karam| 23|   450|     young|             540.0|
|khaled| 16|   220|  teenager|             264.0|
|  Omar| 31|   520|     adult|             624.0|
| Ahmad| 17|   250|  teenager|             300.0|
|   Ali| 22|   410|     young|             492.0|
|  Hasn| 10|     0|     child|               0.0|
+------+---+------+----------+------------------+



###### Filtering

In [109]:
# filter # display data is age greater than 20 # 1
df_1.where(col("age")>20).show()

+-----+---+------+----------+
| name|age|salary|life_stage|
+-----+---+------+----------+
|Karam| 23|   450|     young|
| Omar| 31|   520|     adult|
|  Ali| 22|   410|     young|
+-----+---+------+----------+



In [110]:
## filter # display data is age greater than 20 # 2 
df_1.filter("age>20").show()

+-----+---+------+----------+
| name|age|salary|life_stage|
+-----+---+------+----------+
|Karam| 23|   450|     young|
| Omar| 31|   520|     adult|
|  Ali| 22|   410|     young|
+-----+---+------+----------+



In [111]:
#filter salary  less than  or equal to 350

df_1.filter("salary<=350").show()

+------+---+------+----------+
|  name|age|salary|life_stage|
+------+---+------+----------+
|khaled| 16|   220|  teenager|
| Ahmad| 17|   250|  teenager|
|  Hasn| 10|     0|     child|
+------+---+------+----------+



In [112]:
df_1.where(col("salary").between(300,550)).show()

+-----+---+------+----------+
| name|age|salary|life_stage|
+-----+---+------+----------+
|Karam| 23|   450|     young|
| Omar| 31|   520|     adult|
|  Ali| 22|   410|     young|
+-----+---+------+----------+



In [113]:
#filters DataFrame to include only rows where 'life_stage' is "teenager" or "adult"
df_1.where(col('life_stage').isin(['teenager' ,'adult'])).show()

+------+---+------+----------+
|  name|age|salary|life_stage|
+------+---+------+----------+
|khaled| 16|   220|  teenager|
|  Omar| 31|   520|     adult|
| Ahmad| 17|   250|  teenager|
+------+---+------+----------+



In [114]:
## Filter DataFrame to show rows where the 'name' column ends with "d"

df_1.where(col("name").endswith("d")).show()

+------+---+------+----------+
|  name|age|salary|life_stage|
+------+---+------+----------+
|khaled| 16|   220|  teenager|
| Ahmad| 17|   250|  teenager|
+------+---+------+----------+



###### Grouping and Aggregation:

In [115]:
df_1.groupBy("life_stage").mean().show()

+----------+--------+-----------+
|life_stage|avg(age)|avg(salary)|
+----------+--------+-----------+
|     young|    22.5|      430.0|
|  teenager|    16.5|      235.0|
|     adult|    31.0|      520.0|
|     child|    10.0|        0.0|
+----------+--------+-----------+



###### SQL Queries

In [116]:
#get name dataframe
df_1.columns

['name', 'age', 'salary', 'life_stage']

In [117]:
spark.sql("select avg(salary) from {df1}",df1=df_1).show()

+-----------------+
|      avg(salary)|
+-----------------+
|308.3333333333333|
+-----------------+



In [118]:
spark.sql("select age,salary from {df_1}",df_1=df_1).show()

+---+------+
|age|salary|
+---+------+
| 23|   450|
| 16|   220|
| 31|   520|
| 17|   250|
| 22|   410|
| 10|     0|
+---+------+



In [125]:
spark.sql("select age,salary+10 from {df_1}",df_1=df_1).show()

+---+-------------+
|age|(salary + 10)|
+---+-------------+
| 23|          460|
| 16|          230|
| 31|          530|
| 17|          260|
| 22|          420|
| 10|           10|
+---+-------------+



In [152]:
#function executes an SQL query 
def query_sql(query):
    if len(query)>10:
        spark.sql(str(query),df_1=df_1).show()
    else:
        print("check query and again")

In [153]:
query_sql("select * from {df_1}")

+------+---+------+----------+
|  name|age|salary|life_stage|
+------+---+------+----------+
| Karam| 23|   450|     young|
|khaled| 16|   220|  teenager|
|  Omar| 31|   520|     adult|
| Ahmad| 17|   250|  teenager|
|   Ali| 22|   410|     young|
|  Hasn| 10|     0|     child|
+------+---+------+----------+



In [140]:
query_sql("select avg(salary) from {df_1}")

+-----------------+
|      avg(salary)|
+-----------------+
|308.3333333333333|
+-----------------+



In [154]:
query_sql("")

check query and again


In [155]:
query_sql("select * from {df_1} where salary>=350")

+-----+---+------+----------+
| name|age|salary|life_stage|
+-----+---+------+----------+
|Karam| 23|   450|     young|
| Omar| 31|   520|     adult|
|  Ali| 22|   410|     young|
+-----+---+------+----------+



In [156]:
query_sql("select life_stage,salary from {df_1} where age>25")

+----------+------+
|life_stage|salary|
+----------+------+
|     adult|   520|
+----------+------+

